In [92]:
%%writefile app.py
from itertools import product
import pandas as pd
import psycopg2
import streamlit as st
import plotly.express as px
#import pass_file
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


from wordcloud import WordCloud

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem import SnowballStemmer
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import nltk
nltk.download('punkt')
nltk.download('stopwords')






Overwriting app.py


In [93]:
%%writefile -a app.py

def get_data(cursor, query):
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data)
    return df

Appending to app.py


In [94]:
%%writefile -a app.py

st.title('Summary Dashboard')


Appending to app.py


In [95]:
%%writefile -a app.py
#establishing the connection
conn = psycopg2.connect(database = "Yelp", user = "postgres", password = "root", host = "127.0.0.1", port = "5432")
print ("Opened database successfully")
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

Appending to app.py


In [96]:
%%writefile -a app.py
class BubbleChart:
    def __init__(self, area, bubble_spacing=0):
        """
        Setup for bubble collapse.

        Parameters
        ----------
        area : array-like
            Area of the bubbles.
        bubble_spacing : float, default: 0
            Minimal spacing between bubbles after collapsing.

        Notes
        -----
        If "area" is sorted, the results might look weird.
        """
        area = np.asarray(area)
        r = np.sqrt(area / np.pi)

        self.bubble_spacing = bubble_spacing
        self.bubbles = np.ones((len(area), 4))
        self.bubbles[:, 2] = r
        self.bubbles[:, 3] = area
        self.maxstep = 2 * self.bubbles[:, 2].max() + self.bubble_spacing
        self.step_dist = self.maxstep / 2

        # calculate initial grid layout for bubbles
        length = np.ceil(np.sqrt(len(self.bubbles)))
        grid = np.arange(length) * self.maxstep
        gx, gy = np.meshgrid(grid, grid)
        self.bubbles[:, 0] = gx.flatten()[:len(self.bubbles)]
        self.bubbles[:, 1] = gy.flatten()[:len(self.bubbles)]

        self.com = self.center_of_mass()

    def center_of_mass(self):
        return np.average(
            self.bubbles[:, :2], axis=0, weights=self.bubbles[:, 3]
        )

    def center_distance(self, bubble, bubbles):
        return np.hypot(bubble[0] - bubbles[:, 0],
                        bubble[1] - bubbles[:, 1])

    def outline_distance(self, bubble, bubbles):
        center_distance = self.center_distance(bubble, bubbles)
        return center_distance - bubble[2] - \
            bubbles[:, 2] - self.bubble_spacing

    def check_collisions(self, bubble, bubbles):
        distance = self.outline_distance(bubble, bubbles)
        return len(distance[distance < 0])

    def collides_with(self, bubble, bubbles):
        distance = self.outline_distance(bubble, bubbles)
        idx_min = np.argmin(distance)
        return idx_min if type(idx_min) == np.ndarray else [idx_min]

    def collapse(self, n_iterations=50):
        """
        Move bubbles to the center of mass.

        Parameters
        ----------
        n_iterations : int, default: 50
            Number of moves to perform.
        """
        for _i in range(n_iterations):
            moves = 0
            for i in range(len(self.bubbles)):
                rest_bub = np.delete(self.bubbles, i, 0)
                # try to move directly towards the center of mass
                # direction vector from bubble to the center of mass
                dir_vec = self.com - self.bubbles[i, :2]

                # shorten direction vector to have length of 1
                dir_vec = dir_vec / np.sqrt(dir_vec.dot(dir_vec))

                # calculate new bubble position
                new_point = self.bubbles[i, :2] + dir_vec * self.step_dist
                new_bubble = np.append(new_point, self.bubbles[i, 2:4])

                # check whether new bubble collides with other bubbles
                if not self.check_collisions(new_bubble, rest_bub):
                    self.bubbles[i, :] = new_bubble
                    self.com = self.center_of_mass()
                    moves += 1
                else:
                    # try to move around a bubble that you collide with
                    # find colliding bubble
                    for colliding in self.collides_with(new_bubble, rest_bub):
                        # calculate direction vector
                        dir_vec = rest_bub[colliding, :2] - self.bubbles[i, :2]
                        dir_vec = dir_vec / np.sqrt(dir_vec.dot(dir_vec))
                        # calculate orthogonal vector
                        orth = np.array([dir_vec[1], -dir_vec[0]])
                        # test which direction to go
                        new_point1 = (self.bubbles[i, :2] + orth *
                                      self.step_dist)
                        new_point2 = (self.bubbles[i, :2] - orth *
                                      self.step_dist)
                        dist1 = self.center_distance(
                            self.com, np.array([new_point1]))
                        dist2 = self.center_distance(
                            self.com, np.array([new_point2]))
                        new_point = new_point1 if dist1 < dist2 else new_point2
                        new_bubble = np.append(new_point, self.bubbles[i, 2:4])
                        if not self.check_collisions(new_bubble, rest_bub):
                            self.bubbles[i, :] = new_bubble
                            self.com = self.center_of_mass()

            if moves / len(self.bubbles) < 0.1:
                self.step_dist = self.step_dist / 2

    def plot(self, ax, labels,colors):
        """
        Draw the bubble plot.

        Parameters
        ----------
        ax : matplotlib.axes.Axes
        labels : list
            Labels of the bubbles.
        colors : list
            Colors of the bubbles.
        """
        for i in range(len(self.bubbles)):
            circ = plt.Circle(
                self.bubbles[i, :2], self.bubbles[i, 2], color=colors[i])
            ax.add_patch(circ)
            ax.text(*self.bubbles[i, :2], labels[i],
                    horizontalalignment='center', verticalalignment='center')

Appending to app.py


In [97]:
%%writefile -a app.py
import matplotlib.colors as mcolors
colors = []
for i in mcolors.CSS4_COLORS:
    colors.append(i)
    

Appending to app.py


In [98]:
%%writefile -a app.py

'''
# Max business indexed 
Plot a bubble plot of states with the most businesses indexed on Yelp data

'''

data = [10,20,30,40,50,60,70,80,90,100]
df_n = pd.DataFrame(data, columns=['Numbers'])

top_n = st.selectbox('Choose top "n" Cities', df_n,key=1)
top_n_cities_query = '''SELECT 
    state, city, COUNT(*) AS business_count
FROM
    yelp_business
GROUP BY state , city
ORDER BY business_count DESC
LIMIT {};'''.format(top_n)
cursor.execute(top_n_cities_query)
data = cursor.fetchall()
column_names = ["state","city","count"]
top_n_cities = pd.DataFrame(data,columns = column_names)


top_n_cities['count'] = top_n_cities['count'].apply(pd.to_numeric)
top_n_cities['city'] = top_n_cities['city'].astype(str)
top_n_cities['state'] = top_n_cities['state'].astype(str)


bubble_chart = BubbleChart(area=top_n_cities['count'], bubble_spacing=0.1)
bubble_chart.collapse()

fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
fig.set_size_inches(9, 13, forward=True)
bubble_chart.plot(ax, top_n_cities['state'],colors)
ax.axis("off")
ax.relim()
ax.autoscale_view()
st.pyplot(fig)


Appending to app.py


In [99]:
%%writefile -a app.py
import folium
from streamlit_folium import folium_static 

'''
# Map
Get a locations of a category of business in a city on a map

'''
#list of states and city with drop down for business map
all_state_query = "select distinct state from yelp_business"
cursor.execute(all_state_query)
data = cursor.fetchall()
column_names = ["state"]
all_state = pd.DataFrame(data,columns = column_names)
state_select = st.selectbox('Choose state', all_state,key=11)
if state_select != "":
    #st.write('You selected ' + state_select)
    state_select =  "\'" +state_select + "\'" 
    state_city_query = "select distinct city from yelp_business where state = {}".format(state_select)
    cursor.execute(state_city_query)
    data = cursor.fetchall()
    column_names = ["city"]
    state_city = pd.DataFrame(data,columns = column_names)
    #st.write('state city ' + state_city_query)
    city_select = st.selectbox('Choose city', state_city,key=12)
    #st.write('city selected ' + city_select)
    city_select =  "\'" +city_select + "\'"
    if city_select != "" :
        city_caterories_query = '''
        SELECT distinct C.categories
        FROM yelp_business as B INNER JOIN yelp_categories as C 
        ON B.business_id = C.business_id where B.city = {} order by C.categories limit 20;'''.format(city_select)
        cursor.execute(city_caterories_query)
        data = cursor.fetchall()
        column_names = ["categories"]
        city_caterories = pd.DataFrame(data,columns = column_names)
        #st.write('caterogies ' + city_caterories)
        city_category_select = st.selectbox('Choose category', city_caterories,key=13)
        #st.write('category selected ' + city_category_select)
        city_category_select =  "\'" +city_category_select + "\'"

        if city_category_select !="":
            get_category = '''
            SELECT B.name, B.latitude, B.longitude, C.categories,B.city,B.state,B.review_count
            FROM yelp_business as B INNER JOIN yelp_categories as C 
            ON B.business_id = C.business_id where B.city = {0} and C.categories = {1} ;
            '''.format(city_select,city_category_select)
            cursor.execute(get_category)
            data = cursor.fetchall()
            column_names = ["name","latitude","longitude","categories","city","state","review_count"]
            all_categories_df = pd.DataFrame(data,columns = column_names)
            #st.write(all_categories_df)
            
            map_cat = folium.Map(location=[all_categories_df.latitude.mean(), all_categories_df.longitude.mean()], zoom_start=12,control_scale = True,prefer_canvas=True)
            folium.TileLayer('cartodbpositron').add_to(map_cat)
            for index, location_info in all_categories_df.iterrows():
                info = 'name: {}, review: {}'.format(location_info["name"].strip('\"'), location_info["review_count"])

                folium.Marker([location_info["latitude"], location_info["longitude"]], popup=info).add_to(map_cat)
            
            st.title('Map of ' + city_category_select + 'in' +  city_select)
            folium_static(map_cat)
            
#             st.map(all_categories_df)







Appending to app.py


In [100]:
%%writefile -a app.py

def extract_positive(positive_reviews):
    global positive
    words = [word.lower() for word in word_tokenize(positive_reviews) if word.lower() not in stopwords.words("english") and word.lower().isalpha()]
    positive=positive+words

Appending to app.py


In [101]:
%%writefile -a app.py

def extract_negative(negative_reviews):
    global negative
    words = [word.lower() for word in word_tokenize(negative_reviews) if word.lower() not in stopwords.words("english") and word.lower().isalpha()]
    negative=negative+words

Appending to app.py


In [102]:
%%writefile -a app.py
'''
# Reviews
Get a list of reviews for business of a category in a city


'''
#Get stars and review
#st.write("Stars and Reviews")
#list of states and city with drop down for business map
all_state_query = "select distinct state from yelp_business"
cursor.execute(all_state_query)
data = cursor.fetchall()
column_names = ["state"]
all_state = pd.DataFrame(data,columns = column_names)
state_select = st.selectbox('Choose state', all_state,key=21)
if state_select != "":
    #st.write('You selected ' + state_select)
    state_select =  "\'" +state_select + "\'" 
    state_city_query = "select distinct city from yelp_business where state = {}".format(state_select)
    cursor.execute(state_city_query)
    data = cursor.fetchall()
    column_names = ["city"]
    state_city = pd.DataFrame(data,columns = column_names)
    #st.write('state city ' + state_city_query)
    city_select = st.selectbox('Choose city', state_city,key=22)
    #st.write('city selected ' + city_select)
    city_select =  "\'" +city_select + "\'"
    if city_select != "" :
        city_caterories_query = '''
        SELECT distinct C.categories
        FROM yelp_business as B INNER JOIN yelp_categories as C 
        ON B.business_id = C.business_id where B.city = {} order by C.categories limit 20;'''.format(city_select)
        cursor.execute(city_caterories_query)
        data = cursor.fetchall()
        column_names = ["categories"]
        city_caterories = pd.DataFrame(data,columns = column_names)
        #st.write('caterogies ' + city_caterories)
        city_category_select = st.selectbox('Choose category', city_caterories,key=23)
        #st.write('category selected ' + city_category_select)
        city_category_select =  "\'" +city_category_select + "\'"

        if city_category_select !="":
            review_star_query = '''SELECT T.text, B.stars FROM yelp_business as B INNER JOIN yelp_categories as C
ON B.business_id = C.business_id inner join yelp_tip as T ON B.business_id = T.business_id where C.categories = {} and B.city = {} order by T.text limit 200;  
            '''.format(city_category_select,city_select)
            cursor = conn.cursor()
            cursor.execute(review_star_query)
            data = cursor.fetchall()
            column_names = ["review", "stars"]
            city_caterories = pd.DataFrame(data,columns = column_names)
            #st.table(city_caterories)
                            
            #wordcloud
            if not city_caterories.empty:           
                #wordcloud
                list_review=[]
                for i in city_caterories['review']:
                    list_review.append(len(i))
                city_caterories['text_length']=list_review

                city_caterories['stars']=np.where(city_caterories['stars']>3,1,0)

                positive=[]
                negative=[]
                positive_reviews=city_caterories[city_caterories['stars']==1]['review']
                negative_reviews=city_caterories[city_caterories['stars']==0]['review']

                positive_reviews.apply(extract_positive)
                negative_reviews.apply(extract_negative)

                pos_review_cloud=WordCloud(width=600,height=400).generate(" ".join(positive_reviews))
                if pos_review_cloud != "": 
                    fig, ax = plt.subplots(figsize = (10, 8),facecolor='k')
                    ax.imshow(pos_review_cloud)
                    plt.axis("off")
                    plt.tight_layout(pad=0)
                    st.write('Positive Reviews Word Cloud ')
                    st.pyplot(fig)


                    #plt.figure(figsize=(10,8),facecolor='k')
                    #plt.imshow(pos_review_cloud)
                    #plt.axis('off')
                    #plt.tight_layout(pad=0)
                    #st.pyplot()
                neg_review_cloud=WordCloud(width=600,height=400,background_color='white').generate(" ".join(negative_reviews))
                if neg_review_cloud !="":
                    plt.figure(figsize=(10,8),facecolor='k')
                    fig2, ax = plt.subplots(figsize = (10, 8),facecolor='k')
                    ax.imshow(neg_review_cloud)
                    plt.axis("off")
                    plt.tight_layout(pad=0)       
                    st.write('Negative Reviews Word Cloud ')
                    st.pyplot(fig2)

                    #plt.imshow(neg_review_cloud)
                    #plt.axis('off')
                    #plt.tight_layout(pad=0)
                    #plt.show()


Appending to app.py


In [103]:
%%writefile -a app.py
'''
# Business oppotunities
We will list down categories which have only one business instance in the selected city


'''
#business opportunity
#st.write("business opportunities")
#list of states and city with drop down for business map
all_state_query = "select distinct state from yelp_business"
cursor.execute(all_state_query)
data = cursor.fetchall()
column_names = ["state"]
all_state = pd.DataFrame(data,columns = column_names)
state_select = st.selectbox('Choose state', all_state,key=31)
if state_select != "":
    #st.write('You selected ' + state_select)
    state_select =  "\'" +state_select + "\'" 
    state_city_query = "select distinct city from yelp_business where state = {}".format(state_select)
    cursor.execute(state_city_query)
    data = cursor.fetchall()
    column_names = ["city"]
    state_city = pd.DataFrame(data,columns = column_names)
    #st.write('state city ' + state_city_query)
    city_select = st.selectbox('Choose city', state_city,key=32)
    #st.write('city selected ' + city_select)
    city_select =  "\'" +city_select + "\'"
    if city_select != "" :
#         city_caterories_query = '''
#         SELECT distinct C.categories
#         FROM yelp_business as B INNER JOIN yelp_categories as C 
#         ON B.business_id = C.business_id where B.city = {} order by C.categories limit 20;'''.format(city_select)
#         cursor.execute(city_caterories_query)
#         data = cursor.fetchall()
#         column_names = ["categories"]
#         city_caterories = pd.DataFrame(data,columns = column_names)
#         #st.write('caterogies ' + city_caterories)
#         city_category_select = st.selectbox('Choose category', city_caterories,key=33)
#         #st.write('category selected ' + city_category_select)
#         city_category_select =  "\'" +city_category_select + "\'"

#         if city_category_select !="":
        catogery_count1_query = "SELECT cat, COUNT(cat) FROM (SELECT C.categories as cat FROM yelp_business as B INNER JOIN yelp_categories as C ON B.business_id = C.business_id where B.city = {}) as temp GROUP BY cat having count(cat) =1;".format(city_select)
        cursor = conn.cursor()
        cursor.execute(catogery_count1_query)
        data = cursor.fetchall()
        column_names = ["cat","count"]
        catogery_count1 = pd.DataFrame(data,columns = column_names)
        st.dataframe(catogery_count1["cat"])



Appending to app.py


In [104]:
%%writefile -a app.py

#SQL Query input 
query = st.text_input('Top 15 tips for a selected query')
cursor = conn.cursor()
cursor.execute(query)
data = cursor.fetchall()
result = pd.DataFrame(data)
if not result.empty:
    st.dataframe(result)



Appending to app.py


In [105]:
!streamlit run app.py

^C


In [106]:
# #query = "\"insert into yelp_temp values {} ;\".format('a')"
# query ="'INSERT INTO yelp_temp (name) VALUES("%s")' %('a')"
# cursor = conn.cursor()
# cursor.execute(query)